# Avatar Renderer MCP - Demo Notebook

**Transform static images into dynamic AI-powered avatars with realistic expressions and voice synchronization**

This notebook demonstrates how to:
1. Install and set up Avatar Renderer MCP
2. Generate your first "Hello World" talking avatar
3. Create professional demo videos for marketing
4. Use different quality modes (real-time vs high-quality)

---

**Website:** [https://avatar-renderer-mcp.vercel.app/](https://avatar-renderer-mcp.vercel.app/)

**Repository:** [https://github.com/ruslanmv/avatar-renderer-mcp](https://github.com/ruslanmv/avatar-renderer-mcp)

## 1. Installation & Setup

First, let's install the package and its dependencies.

In [ ]:
# Install uv package manager (if not already installed)
!pip install -q uv

In [ ]:
# Install avatar-renderer-mcp and dependencies
# Note: This assumes you're in the repository root directory
!uv pip install -e .

In [ ]:
# Install external dependencies (SadTalker, Wav2Lip, etc.)
# This may take a few minutes
!make install-git-deps

### 1.1 Configure Environment & Model Paths

**Critical Step**: Set up MODEL_ROOT to point to the local models directory.

In [ ]:
from pathlib import Path
import os

# Verify repository root
repo_root = Path.cwd()
print(f"Repository root: {repo_root}")

# Verify environment is configured correctly
# (These should already be set by the imports cell above)
models_dir = Path(os.environ.get("MODEL_ROOT", repo_root / "models"))
ext_deps_dir = Path(os.environ.get("EXT_DEPS_DIR", repo_root / "external_deps"))

print(f"\n✓ Configuration verified:")
print(f"  MODEL_ROOT   = {models_dir}")
print(f"  EXT_DEPS_DIR = {ext_deps_dir}")
print(f"  Models exist: {models_dir.exists()}")
print(f"  Ext deps exist: {ext_deps_dir.exists()}")

if not models_dir.exists():
    print("\n⚠️  Models directory not found! Run: !make download-models")

In [ ]:
# Download models if not present
!make download-models

## 2. Imports & Configuration

In [ ]:
import os
import sys
from pathlib import Path
from IPython.display import Video, HTML, display
import warnings

warnings.filterwarnings('ignore')

# CRITICAL: Set environment variables BEFORE importing pipeline
# This ensures the pipeline module sees the correct MODEL_ROOT path
repo_root = Path.cwd()
models_dir = repo_root / "models"
ext_deps_dir = repo_root / "external_deps"

os.environ["MODEL_ROOT"] = str(models_dir.resolve())
os.environ["EXT_DEPS_DIR"] = str(ext_deps_dir.resolve())

print(f"✓ Environment configured:")
print(f"  MODEL_ROOT   = {os.environ['MODEL_ROOT']}")
print(f"  EXT_DEPS_DIR = {os.environ['EXT_DEPS_DIR']}")

# Add external dependencies to Python path
if ext_deps_dir.exists():
    sys.path.insert(0, str(ext_deps_dir))
    # Also add subdirectories for external repos (critical for FOMM, SadTalker, Wav2Lip)
    for subdir in ["first-order-model", "SadTalker", "Wav2Lip"]:
        subpath = ext_deps_dir / subdir
        if subpath.exists():
            sys.path.insert(0, str(subpath))
    print(f"✓ Added external dependencies: {ext_deps_dir}")

# NOW import the pipeline (after env vars are set)
from app.pipeline import render_pipeline

print("✓ All imports successful!")

In [ ]:
# Check GPU availability
import torch

cuda_available = torch.cuda.is_available()
print(f"GPU Available: {cuda_available}")
if cuda_available:
    print(f"GPU Device: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️  No GPU detected. Using CPU mode (slower but works).")

## 3. Setup Directories & Assets

In [ ]:
# Create output directory for generated videos
output_dir = Path("demo_outputs")
output_dir.mkdir(exist_ok=True)
print(f"✓ Output directory: {output_dir.absolute()}")

# Check for test assets
test_image = Path("tests/assets/alice.png")
test_audio = Path("tests/assets/hello.wav")

if test_image.exists() and test_audio.exists():
    print(f"✓ Found test assets:")
    print(f"  - Image: {test_image}")
    print(f"  - Audio: {test_audio}")
else:
    print("⚠️  Test assets not found. You'll need to provide your own image and audio files.")

### 3.1 Download Frontend Avatar Presets

Download the same avatar presets used in the frontend website for consistent demos.

In [ ]:
from pathlib import Path
from urllib.parse import quote
import time
import random
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from PIL import Image
from io import BytesIO

# Configure avatar preset directory
AVATAR_DIR = Path("notebook_assets/avatars")
AVATAR_DIR.mkdir(parents=True, exist_ok=True)

POLLINATIONS_BASE = "https://image.pollinations.ai/prompt/"

def pollinations_url(prompt: str, width: int = 768, height: int = 768, nologo: bool = True) -> str:
    """Generate Pollinations AI image URL from prompt"""
    return f"{POLLINATIONS_BASE}{quote(prompt)}?width={width}&height={height}&nologo={'true' if nologo else 'false'}"

# Frontend avatar presets (matching the website)
FRONTEND_PRESETS = [
    {
        "id": "professional",
        "name": "Professional",
        "desc": "Support • Sales • HR",
        "prompt": "professional business portrait photography, studio lighting, neutral background, sharp focus, high detail",
    },
    {
        "id": "creator",
        "name": "Creator",
        "desc": "Influencer • Ads",
        "prompt": "casual portrait of a young content creator, natural lighting, lifestyle photography, friendly smile, high detail",
    },
    {
        "id": "educator",
        "name": "Educator",
        "desc": "Courses • Tutors",
        "prompt": "teacher portrait, friendly professional educator, classroom or education setting, soft lighting, high detail",
    },
    {
        "id": "npc",
        "name": "Game NPC",
        "desc": "Dialogue • Lore",
        "prompt": "3d render game character portrait, fantasy rpg style, detailed, cinematic lighting, centered face",
    },
    {
        "id": "brand",
        "name": "Brand",
        "desc": "Retail • Events",
        "prompt": "brand ambassador professional corporate portrait, confident, clean background, studio lighting, high detail",
    },
    {
        "id": "custom",
        "name": "Custom",
        "desc": "Bring your own",
        "prompt": "futuristic cyberpunk portrait, neon lights, sci-fi style, centered face, high detail",
    },
]

def _requests_session_with_retries() -> requests.Session:
    """
    Create a requests session that retries on common transient HTTP failures
    (including 502 Bad Gateway).
    """
    retry = Retry(
        total=8,
        connect=8,
        read=8,
        backoff_factor=0.8,  # exponential backoff
        status_forcelist=[429, 500, 502, 503, 504],
        allowed_methods=["GET"],
        raise_on_status=False,
    )
    adapter = HTTPAdapter(max_retries=retry)
    s = requests.Session()
    s.mount("http://", adapter)
    s.mount("https://", adapter)
    return s

SESSION = _requests_session_with_retries()

def download_image(url: str, out_path: Path, timeout: int = 120) -> Path:
    """Download image from URL and save as PNG"""
    r = SESSION.get(url, timeout=timeout)
    if r.status_code >= 400:
        # Force a clean error message with status code
        raise requests.HTTPError(f"{r.status_code} {r.reason} for url: {url}", response=r)

    img = Image.open(BytesIO(r.content)).convert("RGB")
    img.save(out_path, format="PNG")
    return out_path

def prepare_frontend_avatars(force_redownload: bool = False):
    """
    Download frontend avatar presets locally.
    Uses fallback sizes if 768 fails (some servers struggle under load).
    """
    local_paths = {}

    # Try big first, then fallback sizes if server is struggling
    size_fallbacks = [(768, 768), (512, 512), (384, 384)]

    for p in FRONTEND_PRESETS:
        preset_id = p["id"]
        out_path = AVATAR_DIR / f"{preset_id}.png"

        if out_path.exists() and not force_redownload:
            print(f"✓ {preset_id} already exists: {out_path}")
            local_paths[preset_id] = str(out_path)
            continue

        print(f"📥 Downloading {preset_id}...")

        last_err = None
        for (w, h) in size_fallbacks:
            url = pollinations_url(p["prompt"], width=w, height=h)

            try:
                # extra manual jitter between requests helps avoid burst rate-limits
                time.sleep(0.3 + random.random() * 0.6)
                download_image(url, out_path, timeout=120)
                print(f"   ✓ Saved to: {out_path}  (size={w}x{h})")
                local_paths[preset_id] = str(out_path)
                last_err = None
                break
            except Exception as e:
                last_err = e
                print(f"   ⚠️ Attempt failed at {w}x{h}: {e}")

        if last_err is not None:
            print(f"   ❌ Failed to download {preset_id} after fallbacks.")
            # continue to next preset, do not kill the notebook

    return local_paths

print("Preparing frontend avatar presets...\n")
avatar_files = prepare_frontend_avatars(force_redownload=False)

print(f"\n✅ Ready! Downloaded {len(avatar_files)} avatar presets:")
for preset_id in avatar_files:
    print(f"   - {preset_id}: {avatar_files[preset_id]}")

## 4. Hello World - Your First Avatar!

Let's create a simple "Hello World" talking avatar. This uses the test assets included in the repository.

In [ ]:
from pathlib import Path
import os, sys
import torch

# --- Hard requirements for runtime ---
# Ensure env vars are correct
REPO_ROOT = Path.cwd()
os.environ.setdefault("MODEL_ROOT", str((REPO_ROOT / "models").resolve()))
os.environ.setdefault("EXT_DEPS_DIR", str((REPO_ROOT / "external_deps").resolve()))

MODEL_ROOT = Path(os.environ["MODEL_ROOT"])
EXT_DEPS_DIR = Path(os.environ["EXT_DEPS_DIR"])

# Ensure external_deps is importable
if str(EXT_DEPS_DIR) not in sys.path:
    sys.path.insert(0, str(EXT_DEPS_DIR))

# --- Check model files ---
has_realtime = (MODEL_ROOT/"sadtalker"/"sadtalker.pth").exists() and (MODEL_ROOT/"wav2lip"/"wav2lip_gan.pth").exists()
has_hq_models = (MODEL_ROOT/"diff2lip"/"Diff2Lip.pth").exists() and (MODEL_ROOT/"fomm"/"vox-cpk.pth").exists()

# --- Check external deps needed for HQ ---
has_fomm_repo = (EXT_DEPS_DIR/"first-order-model").exists()
# ffmpeg-python import test (needed by first-order-model/demo.py)
try:
    import ffmpeg  # noqa: F401
    has_ffmpeg_py = True
except Exception:
    has_ffmpeg_py = False

print("🎬 Generating Hello World avatar...")
print("\nModel availability:")
print(f"  - Real-time (SadTalker + Wav2Lip): {'✓' if has_realtime else '✗'}")
print(f"  - HQ models (FOMM + Diff2Lip):     {'✓' if has_hq_models else '✗'}")
print("HQ runtime deps:")
print(f"  - first-order-model repo:         {'✓' if has_fomm_repo else '✗'}")
print(f"  - ffmpeg-python module:           {'✓' if has_ffmpeg_py else '✗'}")
print(f"  - CUDA available:                 {'✓' if torch.cuda.is_available() else '✗'}")
print(f"\nMODEL_ROOT   = {MODEL_ROOT}")
print(f"EXT_DEPS_DIR = {EXT_DEPS_DIR}")

# --- Choose face image (preset if available) ---
face_image = str(test_image)
if "avatar_files" in globals() and "professional" in avatar_files:
    face_image = avatar_files["professional"]

# --- Render with robust fallback ---
output_video = output_dir / "hello_world.mp4"
print("\nThis may take 30-60 seconds depending on your hardware...\n")

try:
    # Try HQ only if everything required is truly present
    if has_hq_models and has_fomm_repo and has_ffmpeg_py and torch.cuda.is_available():
        print("Using high_quality mode...")
        result = render_pipeline(
            face_image=str(face_image),
            audio=str(test_audio),
            out_path=str(output_video),
            quality_mode="high_quality",
        )
    elif has_realtime:
        print("Using real_time mode...")
        result = render_pipeline(
            face_image=str(face_image),
            audio=str(test_audio),
            out_path=str(output_video),
            quality_mode="real_time",
        )
    else:
        raise RuntimeError(
            "No runnable mode available.\n"
            "Fix: download models and install external deps.\n"
            "Run: make install-git-deps && make download-models"
        )

    print(f"\n✅ Success! Video saved to: {result}")

except Exception as e:
    print(f"❌ Render failed: {e}")
    print("\nTroubleshooting checklist:")
    print("1) External deps installed?  -> make install-git-deps")
    print("2) Models downloaded?        -> make download-models")
    print("3) ffmpeg binary installed?  -> apt-get install ffmpeg (or brew install ffmpeg)")
    print("4) ffmpeg-python installed?  -> pip install ffmpeg-python")
    print("5) If no GPU, use real_time")

In [ ]:
# Display the generated video
if output_video.exists():
    print("🎥 Your first AI avatar:")
    display(Video(str(output_video), width=640, embed=True))
else:
    print("Video file not found. Check the error messages above.")

## 5. Create Custom Demo Videos

Now let's create professional demo videos for your website. You can use your own images and audio files.

In [ ]:
# Helper function to display videos inline
def display_video(video_path, title="Generated Video", width=640):
    """Display video with title"""
    if Path(video_path).exists():
        print(f"\n{'='*60}")
        print(f"🎬 {title}")
        print(f"{'='*60}")
        display(Video(str(video_path), width=width, embed=True))
        file_size = Path(video_path).stat().st_size / (1024 * 1024)  # MB
        print(f"\n📊 File size: {file_size:.2f} MB")
        print(f"📁 Location: {video_path}")
    else:
        print(f"❌ Video not found: {video_path}")

### Example 1: Professional Avatar (Customer Support)

Perfect for:
- Customer support representatives
- Company announcements
- Professional presentations

In [ ]:
from pathlib import Path
import os, sys
import torch

professional_video = output_dir / "professional_demo.mp4"

MODEL_ROOT = Path(os.environ["MODEL_ROOT"])
EXT_DEPS_DIR = Path(os.environ.get("EXT_DEPS_DIR", "external_deps"))

# Ensure external deps importable
if str(EXT_DEPS_DIR) not in sys.path:
    sys.path.insert(0, str(EXT_DEPS_DIR))

has_realtime = (MODEL_ROOT/"sadtalker"/"sadtalker.pth").exists() and (MODEL_ROOT/"wav2lip"/"wav2lip_gan.pth").exists()
has_hq_models = (MODEL_ROOT/"diff2lip"/"Diff2Lip.pth").exists() and (MODEL_ROOT/"fomm"/"vox-cpk.pth").exists()
has_fomm_repo = (EXT_DEPS_DIR/"first-order-model").exists()

try:
    import ffmpeg  # noqa: F401
    has_ffmpeg_py = True
except Exception:
    has_ffmpeg_py = False

print("🎬 Creating professional customer support avatar...\n")
print("Model availability:")
print(f"  - Real-time mode (SadTalker + Wav2Lip): {'✓' if has_realtime else '✗'}")
print(f"  - High-quality models (FOMM + Diff2Lip): {'✓' if has_hq_models else '✗'}")
print("HQ runtime deps:")
print(f"  - first-order-model repo: {'✓' if has_fomm_repo else '✗'}")
print(f"  - ffmpeg-python module:   {'✓' if has_ffmpeg_py else '✗'}")
print(f"  - CUDA available:         {'✓' if torch.cuda.is_available() else '✗'}\n")

# Select face image
if 'avatar_files' in globals() and 'professional' in avatar_files:
    face_image = avatar_files['professional']
    print(f"Using frontend preset avatar: {face_image}")
else:
    face_image = str(test_image)
    print(f"Using test asset: {face_image}")

def try_realtime():
    if not has_realtime:
        raise RuntimeError("Real-time models missing. Run: make download-models")
    return render_pipeline(
        face_image=str(face_image),
        audio=str(test_audio),
        out_path=str(professional_video),
        quality_mode="real_time",
    )

try:
    if has_hq_models and has_fomm_repo and has_ffmpeg_py and torch.cuda.is_available():
        print("\nAttempting high_quality mode...")
        result = render_pipeline(
            face_image=str(face_image),
            audio=str(test_audio),
            out_path=str(professional_video),
            quality_mode="high_quality",
        )
    else:
        print("\nSkipping high_quality (missing GPU or runtime deps). Using real_time...")
        result = try_realtime()

    print(f"\n✅ Success! Video saved to: {result}")
    display_video(result, "Professional Avatar")

except Exception as e:
    print(f"⚠️ High_quality failed (or pipeline error): {e}")
    print("\nFalling back to real_time...")
    try:
        result = try_realtime()
        print(f"\n✅ Success! Video saved to: {result}")
        display_video(result, "Professional Avatar (Real-Time Fallback)")
    except Exception as e2:
        print(f"❌ Fallback also failed: {e2}")
        print("\nTroubleshooting:")
        print(f"1. MODEL_ROOT = {MODEL_ROOT}")
        print(f"2. EXT_DEPS_DIR = {EXT_DEPS_DIR}")
        print("3. Run: make install-git-deps")
        print("4. Run: make download-models")
        print("5. Install: pip install ffmpeg-python")
        print("6. Install system ffmpeg (apt/brew)")

### Example 2: Real-Time Avatar (Live Streaming)

Perfect for:
- Live news broadcasts
- Real-time chatbots
- Interactive virtual assistants

**Target: <3s latency**

In [ ]:
import time
from pathlib import Path
import os, sys

realtime_video = output_dir / "realtime_demo.mp4"

MODEL_ROOT = Path(os.environ["MODEL_ROOT"])
EXT_DEPS_DIR = Path(os.environ.get("EXT_DEPS_DIR", "external_deps"))

# Ensure external deps importable
if str(EXT_DEPS_DIR) not in sys.path:
    sys.path.insert(0, str(EXT_DEPS_DIR))

print("⚡ Creating real-time avatar (optimized for speed)...\n")

# Check if real-time models are available
has_realtime = (MODEL_ROOT/"sadtalker"/"sadtalker.pth").exists() and (MODEL_ROOT/"wav2lip"/"wav2lip_gan.pth").exists()

if not has_realtime:
    print("❌ Real-time models not available!")
    print(f"\nMODEL_ROOT: {MODEL_ROOT}")
    print("\nRequired files:")
    print(f"  - sadtalker/sadtalker.pth: {'✓' if (MODEL_ROOT/'sadtalker'/'sadtalker.pth').exists() else '✗'}")
    print(f"  - wav2lip/wav2lip_gan.pth: {'✓' if (MODEL_ROOT/'wav2lip'/'wav2lip_gan.pth').exists() else '✗'}")
    print("\nPlease run: make download-models")
else:
    # Use preset avatar if available
    if 'avatar_files' in globals() and 'creator' in avatar_files:
        face_image = avatar_files['creator']
        print(f"Using frontend preset avatar: {face_image}\n")
    else:
        face_image = str(test_image)
        print(f"Using test asset: {face_image}\n")
    
    start_time = time.time()
    
    try:
        result = render_pipeline(
            face_image=face_image,
            audio=str(test_audio),
            out_path=str(realtime_video),
            quality_mode="real_time"  # Fast processing for streaming
        )
        
        processing_time = time.time() - start_time
        
        print(f"\n⏱️  Processing time: {processing_time:.2f} seconds")
        display_video(result, "Real-Time Avatar (Live Streaming Ready)")
        
    except Exception as e:
        print(f"❌ Error: {e}")
        print(f"\nMODEL_ROOT: {MODEL_ROOT}")
        print(f"EXT_DEPS_DIR: {EXT_DEPS_DIR}")
        print("\nTroubleshooting:")
        print("1. Ensure models are downloaded: make download-models")
        print("2. Install external deps: make install-git-deps")
        print("3. Check that external_deps/ contains SadTalker and Wav2Lip")

## 6. Quality Mode Comparison

Let's compare the two quality modes:

| Feature | Real-Time | High-Quality |
|---------|-----------|-------------|
| **Speed** | <3s | ~10-30s |
| **GPU Required** | No | Yes |
| **Bitrate** | 2 Mbps | 6 Mbps |
| **Enhancement** | None | GFPGAN |
| **Best For** | Live streaming | Pre-recorded content |
| **Pipeline** | SadTalker + Wav2Lip | FOMM + Diff2Lip + GFPGAN |

## 7. Create Your Own Content

Ready to create your own avatar videos? Here's a template:

In [ ]:
# Template for creating your own avatar videos

def create_custom_avatar(
    image_path: str,
    audio_path: str,
    output_name: str,
    quality: str = "auto",
    description: str = "Custom Avatar"
):
    """
    Create a custom avatar video
    
    Args:
        image_path: Path to avatar image (PNG/JPG)
        audio_path: Path to audio file (WAV/MP3)
        output_name: Name for output video (without extension)
        quality: 'real_time', 'high_quality', or 'auto'
        description: Description for display
    """
    output_path = output_dir / f"{output_name}.mp4"
    
    print(f"🎬 Creating {description}...")
    print(f"   Image: {image_path}")
    print(f"   Audio: {audio_path}")
    print(f"   Quality: {quality}\n")
    
    try:
        result = render_pipeline(
            face_image=image_path,
            audio=audio_path,
            out_path=str(output_path),
            quality_mode=quality
        )
        display_video(result, description)
        return result
    except Exception as e:
        print(f"❌ Error: {e}")
        return None

print("✓ Custom avatar function ready!")
print("\nUsage example:")
print('create_custom_avatar(')
print('    image_path="path/to/your/image.png",')
print('    audio_path="path/to/your/audio.wav",')
print('    output_name="my_avatar",')
print('    quality="high_quality",')
print('    description="My Custom Avatar"')
print(')')

In [ ]:
# Example: Create your own avatar
# Uncomment and modify the paths below:

# create_custom_avatar(
#     image_path="path/to/your/image.png",
#     audio_path="path/to/your/audio.wav",
#     output_name="my_first_avatar",
#     quality="auto",
#     description="My First Custom Avatar"
# )

## 8. Tips for Creating Great Demo Videos

### 🖼️ Image Requirements
- **Format**: PNG or JPG
- **Face**: Clear, front-facing portrait
- **Resolution**: Minimum 512x512, recommended 1024x1024
- **Lighting**: Well-lit, avoid harsh shadows
- **Background**: Clean, uncluttered

### 🎤 Audio Requirements
- **Format**: WAV or MP3
- **Sample Rate**: 16kHz recommended
- **Quality**: Clear speech, minimal background noise
- **Length**: Keep it concise (10-30 seconds for demos)

### 🎬 Demo Video Best Practices
1. **Keep it short**: 5-15 seconds for website demos
2. **Clear message**: Focus on one key benefit
3. **Professional appearance**: Use high-quality mode for marketing
4. **Multiple variations**: Create videos for different use cases
5. **Test on different devices**: Ensure compatibility

### 📊 Recommended Demo Topics
1. **Customer Support**: "Hello! I'm here to help you 24/7"
2. **AI Teacher**: "Welcome to today's lesson on..."
3. **Brand Ambassador**: "Discover our latest product..."
4. **Game NPC**: "Greetings, traveler! What brings you here?"
5. **News Anchor**: "Good evening, here are today's top stories..."

## 9. Batch Processing (Multiple Videos)

Create multiple demo videos at once for your website:

In [ ]:
def batch_create_demos(demos_config):
    """
    Create multiple demo videos from a configuration list
    
    Args:
        demos_config: List of dictionaries with demo configurations
    """
    results = []
    
    print(f"🎬 Starting batch creation of {len(demos_config)} videos...\n")
    
    for i, config in enumerate(demos_config, 1):
        print(f"\n{'='*60}")
        print(f"Processing {i}/{len(demos_config)}: {config['name']}")
        print(f"{'='*60}")
        
        result = create_custom_avatar(
            image_path=config['image'],
            audio_path=config['audio'],
            output_name=config['output'],
            quality=config.get('quality', 'auto'),
            description=config['name']
        )
        
        results.append({
            'name': config['name'],
            'output': result,
            'success': result is not None
        })
    
    # Summary
    print(f"\n\n{'='*60}")
    print("📊 BATCH PROCESSING SUMMARY")
    print(f"{'='*60}")
    
    successful = sum(1 for r in results if r['success'])
    print(f"✅ Successful: {successful}/{len(results)}")
    
    if successful < len(results):
        print(f"❌ Failed: {len(results) - successful}")
        print("\nFailed videos:")
        for r in results:
            if not r['success']:
                print(f"  - {r['name']}")
    
    return results

# Example configuration for multiple demos
demo_configs = [
    {
        'name': 'Hello World Demo',
        'image': str(test_image),
        'audio': str(test_audio),
        'output': 'demo_1_hello',
        'quality': 'auto'
    },
    # Add more demos here:
    # {
    #     'name': 'Customer Support Demo',
    #     'image': 'assets/support_avatar.png',
    #     'audio': 'assets/support_script.wav',
    #     'output': 'demo_2_support',
    #     'quality': 'high_quality'
    # },
]

# Uncomment to run batch processing:
# results = batch_create_demos(demo_configs)

### 9.1 Render All Frontend Preset Avatars

Create videos for all downloaded frontend avatar presets (matching the website).

In [ ]:
# Render ALL frontend preset avatars
# Uncomment to execute batch rendering

# from pathlib import Path
#
# if 'avatar_files' in globals() and avatar_files:
#     print(f"🎬 Batch rendering {len(avatar_files)} frontend preset avatars...\n")
#     
#     outputs = {}
#     for preset_id, face_image in avatar_files.items():
#         out_path = output_dir / f"preset_{preset_id}.mp4"
#         
#         print(f"\n{'='*60}")
#         print(f"Rendering preset: {preset_id}")
#         print(f"{'='*60}")
#         
#         try:
#             result = render_pipeline(
#                 face_image=face_image,
#                 audio=str(test_audio),
#                 out_path=str(out_path),
#                 quality_mode="auto",  # Auto-select based on available models
#             )
#             outputs[preset_id] = result
#             print(f"✓ Success: {result}")
#         except Exception as e:
#             print(f"✗ Failed: {e}")
#             outputs[preset_id] = None
#     
#     # Summary
#     print(f"\n\n{'='*60}")
#     print("📊 BATCH RENDERING SUMMARY")
#     print(f"{'='*60}")
#     
#     successful = sum(1 for v in outputs.values() if v is not None)
#     print(f"\n✅ Successful: {successful}/{len(outputs)}")
#     
#     if successful < len(outputs):
#         print(f"❌ Failed: {len(outputs) - successful}")
#         print("\nFailed presets:")
#         for k, v in outputs.items():
#             if v is None:
#                 print(f"  - {k}")
#     
#     print(f"\nOutput files in: {output_dir}/")
# else:
#     print("⚠️ No avatar presets available. Run the avatar download cell first.")

## 10. API Reference

### Core Function: `render_pipeline()`

```python
render_pipeline(
    face_image: str,           # Path to avatar image (PNG/JPG)
    audio: str,                # Path to audio file (WAV/MP3)
    out_path: str,             # Output video path
    reference_video: str = None,  # Optional driving video
    quality_mode: str = "auto"    # "real_time", "high_quality", or "auto"
) -> str  # Returns absolute path to generated video
```

### Quality Modes

#### `real_time`
- **Speed**: <3 seconds latency
- **Pipeline**: SadTalker + Wav2Lip
- **GPU**: Optional (CPU fallback available)
- **Use cases**: Live streaming, chatbots, real-time interactions

#### `high_quality`
- **Quality**: Maximum with GFPGAN enhancement
- **Pipeline**: FOMM + Diff2Lip + GFPGAN
- **GPU**: Required (V100 or better recommended)
- **Use cases**: YouTube content, marketing videos, professional productions

#### `auto` (default)
- Automatically selects the best mode based on:
  - GPU availability
  - Model checkpoint availability
  - System resources

## 11. Next Steps

### 📚 Learn More
- Read the full documentation: [README.md](README.md)
- Explore quality modes: [docs/QUALITY_MODES.md](docs/QUALITY_MODES.md)
- Check out the API: Start the FastAPI server with `make run`

### 🚀 Deploy to Production
1. **Docker**: `make docker-build && make docker-run`
2. **Kubernetes**: `helm install avatar-renderer ./charts/avatar-renderer`
3. **Frontend**: Deploy the Next.js app from `/frontend` to Vercel

### 🎨 Create More Content
1. Prepare your avatar images and scripts
2. Use this notebook to generate demo videos
3. Upload to your website at [https://avatar-renderer-mcp.vercel.app/](https://avatar-renderer-mcp.vercel.app/)

### 💡 Advanced Features
- **MCP Integration**: Use the STDIO server for AI agent communication
- **REST API**: Integrate with your applications via FastAPI
- **Custom Models**: Fine-tune models for your specific use case
- **Batch Processing**: Process multiple videos efficiently

---

## 🎉 Congratulations!

You've successfully created your first AI-powered talking avatar!

**Need help?** Check out:
- 🌐 Website: [https://avatar-renderer-mcp.vercel.app/](https://avatar-renderer-mcp.vercel.app/)
- 📦 Repository: [https://github.com/ruslanmv/avatar-renderer-mcp](https://github.com/ruslanmv/avatar-renderer-mcp)
- 📧 Contact: contact@ruslanmv.com
- 🌟 Author: [Ruslan Magana Vsevolodovna](https://ruslanmv.com)

---

*Transform static images into dynamic, AI-powered avatars with realistic expressions and voice synchronization*